In [2]:
%%time
from multiprocessing import Pool
import re, os, json
from string import digits
import pandas as pd

docIDs = list(range(0, 18))

def delete_sings_digits(content):
    punc = '''!()-'”№[]{};:"\,“«»<>./?@#$%—…^&*_~|–abcdefghijklmnoqprstuvwxyz'''
    content = "".join([ele for ele in content.lower() if ele not in punc])
    table = str.maketrans('', '', digits)
    content = content.translate(table)
    return content


def create_vocabulary(content):    
    raw_content = delete_sings_digits(content)
    vocab = [i.lower() for i in raw_content.split() if i != '']
    return vocab

def read_book(book):
    dirname='../books/'
#     tree = Tree()
    with open(dirname + book, 'r', encoding='utf-8') as file:
        text = create_vocabulary(file.read())
#     unique = list(set(text))
#     for key in unique:
#         tree.insert(key)
    return text


def isUkrainian(word):
    ukrainian = [96, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1100, 1102, 1103, 1108, 1110, 1111, 1169, 8217]
    for l in word:
        if ord(l) not in ukrainian:
#             print("NOT UKRAINIAN LETTER:", l, "IN WORD", key)
            return False
    return True
       
        
def get_vocabulary(dirname='../books/'):
    books = os.listdir(dirname)
    vocabulary = list()
    dictionary = dict()
    memory = 0
    with Pool(8) as p:
        vocabul = p.map(read_book, books)
    for i, book in enumerate(books):
        dictionary[book.replace('.txt', '')] = vocabul[i]
        vocabulary += vocabul[i]
        print(book, "is read!")
        memory += os.stat(dirname + book).st_size
    return memory, vocabulary, dictionary


memory, vocabulary, dictionary = get_vocabulary()
print(f'Total number of words : {len(vocabulary)}')
print(f'Total number of tokens : {len(set(vocabulary))}')

# tree = Tree()
ukr_voc = [k for k in vocabulary if isUkrainian(k) == True]

Тигролови.txt is read!
Захар_Беркут.txt is read!
Камінний_хрест.txt is read!
Intermezzo.txt is read!
Хіба_ревуть_воли_як_ясла_повні.txt is read!
Чорна_Рада.txt is read!
Я_романтика.txt is read!
Земля.txt is read!
Кобзар.txt is read!
Лісова_пісня.txt is read!
Місто.txt is read!
Маруся_Чурай.txt is read!
Климко.txt is read!
Зів'яле_листя.txt is read!
Мина_Мазайло.txt is read!
Тіні_забутих_предків.txt is read!
Мартин_Боруля.txt is read!
Кайдашева_сім'я.txt is read!
Total number of words : 708449
Total number of tokens : 94292
CPU times: user 1.47 s, sys: 84.1 ms, total: 1.55 s
Wall time: 1.97 s


In [11]:
len(ukr_voc)

707575

In [3]:
from itertools import zip_longest, islice


def to_int_keys_best(l):
    """
    l: iterable of keys
    returns: a list with integer keys
    """
    seen = set()
    ls = []
    for e in l:
        if not e in seen:
            ls.append(e)
            seen.add(e)
    ls.sort()
    index = {v: i for i, v in enumerate(ls)}
    return [index[v] for v in l]


def suffix_matrix_best(s):
    """
    suffix matrix of s
    O(n * log(n)^2)
    """
    n = len(s)
    k = 1
    line = to_int_keys_best(s)
    ans = [line]
    while max(line) < n - 1:
        line = to_int_keys_best(
            [a * (n + 1) + b + 1
             for (a, b) in
             zip_longest(line, islice(line, k, None),
                         fillvalue=-1)])
        ans.append(line)
        k <<= 1
    return ans


def suffix_array_best(s):
    """
    suffix array of s
    O(n * log(n)^2)
    """
    n = len(s)
    k = 1
    line = to_int_keys_best(s)
    while max(line) < n - 1:
        line = to_int_keys_best(
            [a * (n + 1) + b + 1
             for (a, b) in
             zip_longest(line, islice(line, k, None),
                         fillvalue=-1)])
        k <<= 1
    return line


In [10]:


def suffix_array_alternative_naive(s):
    suffixes = [(suffix, rank) for suffix, rank in sorted((s[i:], i) for i in range(len(s)))]
    for i in range(len(suffixes)):
        print(suffixes[i])
    return [rank for suffix, rank in sorted((s[i:], i) for i in range(len(s)))]

suffix_array_alternative_naive('banana')



('a', 5)
('ana', 3)
('anana', 1)
('banana', 0)
('na', 4)
('nana', 2)


[5, 3, 1, 0, 4, 2]

In [6]:
suffix_matrix_best('banana')

[[1, 0, 2, 0, 2, 0], [2, 1, 3, 1, 3, 0], [3, 2, 5, 1, 4, 0]]